In [2]:
import pandas as pd
import numpy as np

!pip install tqdm
!pip install matplotlib

In [3]:
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


# <span style="color:red">A POKEMON DATASET </span>
## USING EXTERNAL APIS TO READ OBJECTS

In [5]:
url= "https://pokeapi.co/api/v2/pokemon-species?limit=2000"

In [6]:
res = requests.get(url)
data = res.json()

In [7]:
df = pd.DataFrame(data['results'])
df


,name,url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon-species/1/
1,ivysaur,https://pokeapi.co/api/v2/pokemon-species/2/
2,venusaur,https://pokeapi.co/api/v2/pokemon-species/3/
3,charmander,https://pokeapi.co/api/v2/pokemon-species/4/
4,charmeleon,https://pokeapi.co/api/v2/pokemon-species/5/
...,...,...
1020,raging-bolt,https://pokeapi.co/api/v2/pokemon-species/1021/
1021,iron-boulder,https://pokeapi.co/api/v2/pokemon-species/1022/
1022,iron-crown,https://pokeapi.co/api/v2/pokemon-species/1023/
1023,terapagos,https://pokeapi.co/api/v2/pokemon-species/1024/


## Looping the details to get pokemons

In [8]:
details = []
for u in tqdm(df["url"]):
    poke_data = requests.get(u).json()


    details.append({
        "name": poke_data["name"],
        "id":poke_data["id"],
        "base_happiness": poke_data["base_happiness"],
        "capture_rate": poke_data["capture_rate"],
        "gender_rate": poke_data["gender_rate"],
        "generation": poke_data["generation"]["name"],
        "growth_rate": poke_data["growth_rate"]["name"],
        "habitat":poke_data["habitat"]["name"] if poke_data.get("habitat") else None,
        "is_legendary": poke_data["is_legendary"],
        "is_mythical":poke_data["is_mythical"],
    })


100%|██████████| 1025/1025 [01:00<00:00, 16.86it/s]


In [ ]:
df2 = pd.DataFrame(details)
df2

,name,id,base_happiness,capture_rate,gender_rate,generation,growth_rate,habitat,is_legendary,is_mythical
0,bulbasaur,1,70,45,1,generation-i,medium-slow,grassland,False,False
1,ivysaur,2,70,45,1,generation-i,medium-slow,grassland,False,False
2,venusaur,3,70,45,1,generation-i,medium-slow,grassland,False,False
3,charmander,4,70,45,1,generation-i,medium-slow,mountain,False,False
4,charmeleon,5,70,45,1,generation-i,medium-slow,mountain,False,False
...,...,...,...,...,...,...,...,...,...,...
1020,raging-bolt,1021,0,10,-1,generation-ix,slow,None,False,False
1021,iron-boulder,1022,0,10,-1,generation-ix,slow,None,False,False
1022,iron-crown,1023,0,10,-1,generation-ix,slow,None,False,False
1023,terapagos,1024,50,255,-1,generation-ix,slow,None,True,False


## Merging the dfs


In [1]:
df = pd.merge(df, df2)
df.set_index("id", inplace=True)
df

NameError: name 'pd' is not defined

Since after gen-3 pokemon dont have their habital information given in apis. We are going to use their generation to assign their region to their habitat

In [ ]:
generation_to_region = {
    "generation-i": "Kanto",
    "generation-ii": "Johto",
    "generation-iii": "Hoenn",
    "generation-iv": "Sinnoh",
    "generation-v": "Unova",
    "generation-vi": "Kalos",
    "generation-vii": "Alola",
    "generation-viii": "Galar",
    "generation-ix": "Paldea"
}
df.loc[df["habitat"].isna(), "habitat"] = df.loc[df["habitat"].isna(), "generation"].map(generation_to_region)

df["hngbd"] = (df["generation"].map(generation_to_region) == df["habitat"]) #habitat not given by default
df

,name,url,base_happiness,capture_rate,gender_rate,generation,growth_rate,habitat,is_legendary,is_mythical,hngbd
id,,,,,,,,,,,
1,bulbasaur,https://pokeapi.co/api/v2/pokemon-species/1/,70,45,1,generation-i,medium-slow,grassland,False,False,False
2,ivysaur,https://pokeapi.co/api/v2/pokemon-species/2/,70,45,1,generation-i,medium-slow,grassland,False,False,False
3,venusaur,https://pokeapi.co/api/v2/pokemon-species/3/,70,45,1,generation-i,medium-slow,grassland,False,False,False
4,charmander,https://pokeapi.co/api/v2/pokemon-species/4/,70,45,1,generation-i,medium-slow,mountain,False,False,False
5,charmeleon,https://pokeapi.co/api/v2/pokemon-species/5/,70,45,1,generation-i,medium-slow,mountain,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
1021,raging-bolt,https://pokeapi.co/api/v2/pokemon-species/1021/,0,10,-1,generation-ix,slow,Paldea,False,False,True
1022,iron-boulder,https://pokeapi.co/api/v2/pokemon-species/1022/,0,10,-1,generation-ix,slow,Paldea,False,False,True
1023,iron-crown,https://pokeapi.co/api/v2/pokemon-species/1023/,0,10,-1,generation-ix,slow,Paldea,False,False,True


## <span style='color:blue'>***TRAINING THE MODEL***</span>

In [ ]:
df = df.drop('url',axis=1)

In [ ]:
df['type'] = df.apply(
    lambda row: 0 if row['is_legendary'] == True else
                (1 if row['is_mythical'] == True else 2),
    axis=1
)
df

,name,base_happiness,capture_rate,gender_rate,generation,growth_rate,habitat,is_legendary,is_mythical,hngbd,type
id,,,,,,,,,,,
1,bulbasaur,70,45,1,generation-i,medium-slow,grassland,False,False,False,2
2,ivysaur,70,45,1,generation-i,medium-slow,grassland,False,False,False,2
3,venusaur,70,45,1,generation-i,medium-slow,grassland,False,False,False,2
4,charmander,70,45,1,generation-i,medium-slow,mountain,False,False,False,2
5,charmeleon,70,45,1,generation-i,medium-slow,mountain,False,False,False,2
...,...,...,...,...,...,...,...,...,...,...,...
1021,raging-bolt,0,10,-1,generation-ix,slow,Paldea,False,False,True,2
1022,iron-boulder,0,10,-1,generation-ix,slow,Paldea,False,False,True,2
1023,iron-crown,0,10,-1,generation-ix,slow,Paldea,False,False,True,2


In [ ]:
X= df.drop(['type','is_legendary','is_mythical'], axis=1)
X= pd.get_dummies(X, drop_first=True)

y = df["type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
df

,name,base_happiness,capture_rate,gender_rate,generation,growth_rate,habitat,is_legendary,is_mythical,hngbd,type
id,,,,,,,,,,,
1,bulbasaur,70,45,1,generation-i,medium-slow,grassland,False,False,False,2
2,ivysaur,70,45,1,generation-i,medium-slow,grassland,False,False,False,2
3,venusaur,70,45,1,generation-i,medium-slow,grassland,False,False,False,2
4,charmander,70,45,1,generation-i,medium-slow,mountain,False,False,False,2
5,charmeleon,70,45,1,generation-i,medium-slow,mountain,False,False,False,2
...,...,...,...,...,...,...,...,...,...,...,...
1021,raging-bolt,0,10,-1,generation-ix,slow,Paldea,False,False,True,2
1022,iron-boulder,0,10,-1,generation-ix,slow,Paldea,False,False,True,2
1023,iron-crown,0,10,-1,generation-ix,slow,Paldea,False,False,True,2


In [ ]:
logistic_model = LogisticRegression(solver='lbfgs',max_iter=1000)
logistic_model.fit(X_train,y_train)
preds= logistic_model.predict(X_test)
accuracy = accuracy_score(y_test,preds)
print('Logistic accuracy', accuracy)

Logistic accuracy 0.9365853658536586


In [ ]:
svm_model = SVC(kernel='rbf', C=1000, gamma='auto')
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_preds)
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.9853658536585366


In [ ]:
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=None, random_state=42)
dt_model.fit(X_train, y_train)
dt_preds = dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_preds)
print("Decision Tree Accuracy:", dt_accuracy)

Decision Tree Accuracy: 0.9658536585365853


In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_depth=None,
    random_state=42
)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.9707317073170731


In [ ]:
def predict_by_name(pokemon_names,df, encoder_columns):
    correct_prediction_counter = {
        'Logistic': 0,
        'SVM': 0,
        'Decision Tree': 0,
        'Random Forest': 0
    }
    for pokemon_name in pokemon_names:
        row = df[df['name'] == pokemon_name.lower()]

        if row.empty:
            print(f"{pokemon_name}: Pokémon not found!")
            continue

        row = row.drop(columns=['type'], errors='ignore')

        row_encoded = pd.get_dummies(row)

        row_encoded = row_encoded.reindex(columns=encoder_columns, fill_value=0)

        #predicting
        pred1 = logistic_model.predict(row_encoded)[0]
        pred2 = svm_model.predict(row_encoded)[0]
        pred3 = dt_model.predict(row_encoded)[0]
        pred4 = rf_model.predict(row_encoded)[0]

        actual_type = df.loc[df['name'] == pokemon_name.lower() ,'type'].values[0]

        #Updating counter
        if pred1 == actual_type:
            correct_prediction_counter['Logistic'] += 1
        if pred2 == actual_type:
            correct_prediction_counter['SVM'] += 1
        if pred3 == actual_type:
            correct_prediction_counter['Decision Tree'] += 1
        if pred4 == actual_type:
            correct_prediction_counter['Random Forest'] += 1

        #printing the output
        # print ('\nLogistic:' , {pokemon_name: 'Legendary' if pred1 == 0 else 'Mythical' if pred1 == 1 else 'Normal'}, '✔' if pred1 == actual_type else '✘')
        # print ('\nSVM:' , {pokemon_name: 'Legendary' if pred2 == 0 else 'Mythical' if pred2 == 1 else 'Normal'}, '✔' if pred2 == actual_type else '✘')
        # print ('\nDecision Tree:' , {pokemon_name: 'Legendary' if pred3 == 0 else 'Mythical' if pred3 == 1 else 'Normal'}, '✔' if pred3 == actual_type else '✘')
        # print ('\nRandom Forest:' , {pokemon_name: 'Legendary' if pred4 == 0 else 'Mythical' if pred4 == 1 else 'Normal'}, '✔' if pred4 == actual_type else '✘')
        # print('--------------------------------------------------------------------------------')

    print('\n The Number of types each model got right \n', correct_prediction_counter)

In [ ]:
predict_by_name(df.loc[(df['type'] == 0) | (df['type']==1), 'name'],df,X.columns)


 The Number of types each model got right 
 {'Logistic': 72, 'SVM': 91, 'Decision Tree': 88, 'Random Forest': 88}
